<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/train_multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# define variables
import os
repository_url = 'https://github.com/JonathanLehner/Colab-collection'
repository_name = os.path.basename(repository_url)
loader_path = './products.py'
training_data_url = 'https://github.com/JonathanLehner/Colab-collection/releases/download/0.1/labeled_training_images.zip'
training_filename = os.path.basename(training_data_url)
training_filename_wo_ext = os.path.splitext(training_filename)[0]

In [3]:
print(repository_name)

Colab-collection


In [4]:
# clone Mask-RCNN repo
%cd /content/
import os
from os.path import exists, join, basename

project_name = "Mask_RCNN"
if not exists(project_name):
  # clone and install
  !git clone -q https://github.com/matterport/Mask_RCNN.git
  !cd $project_name && pip install -q -r requirements.txt
  
import sys
sys.path.append(project_name)

/content


In [5]:
%cd /content/Mask_RCNN

/content/Mask_RCNN


In [6]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [7]:
%cd /content/Mask_RCNN

/content/Mask_RCNN


In [0]:
#clean up in case of repetitions
#!rm huerlimann3.zip
#!rm -r huerlimann3
!rm -r __MACOSX
!rm -r /content/Mask_RCNN/datasets

In [9]:
#load and unzip training data
os.system('wget %s'%training_data_url)
os.system('unzip %s'%training_filename)


0

In [10]:
!mkdir /content/Mask_RCNN/datasets
os.system('mv /content/Mask_RCNN/{0} /content/Mask_RCNN/datasets/{1}'.format(training_filename_wo_ext,training_filename_wo_ext))

0

In [0]:
#!rm -r /content/Mask_RCNN/samples/Colab-collection
#print('git clone {0}'.format(repository_url))
#!git clone https://github.com/JonathanLehner/Colab-collection

In [12]:
# load the training script
%cd /content/Mask_RCNN/samples/
os.system('rm -r {0}'.format(repository_name))

print('git clone {0}'.format(repository_url))
os.system('git clone {0}'.format(repository_url))

/content/Mask_RCNN/samples
git clone https://github.com/JonathanLehner/Colab-collection


0

In [13]:
# how to import products.py file 
'''os.system('/content/Mask_RCNN/samples/{0}'.format(repository_name))
products = __import__("products")
print(products)
config = products.ProductConfig()
PRODUCT_DIR = os.path.join(ROOT_DIR, "datasets/"+repository_name)'''



'os.system(\'/content/Mask_RCNN/samples/{0}\'.format(repository_name))\nproducts = __import__("products")\nprint(products)\nconfig = products.ProductConfig()\nPRODUCT_DIR = os.path.join(ROOT_DIR, "datasets/"+repository_name)'

In [14]:
import os
# call script with subprocess
# train
os.chdir('/content/Mask_RCNN/samples/{0}'.format(repository_name))
# !pwd
command_string = 'python3 {0} train --dataset=/content/Mask_RCNN/datasets/{1} --weights=coco --epochs=30'.format(loader_path, training_filename_wo_ext)

import subprocess
import shlex
args = shlex.split(command_string)
print(args)
process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)
#inspiration from https://www.endpoint.com/blog/2015/01/28/getting-realtime-output-using-python
while True:
    output = process.stdout.readline()
    if process.poll() is not None:
        break
    if output != b'':
        print(output)
    rc = process.poll()



['python3', './products.py', 'train', '--dataset=/content/Mask_RCNN/datasets/labeled_training_images', '--weights=coco', '--epochs=30']
b'Using TensorFlow backend.\n'
b'WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.\n'
b'Instructions for updating:\n'
b'Colocations handled automatically by placer.\n'
b'2019-04-30 21:45:43.734067: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz\n'
b'2019-04-30 21:45:43.734414: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17dc4520 executing computations on platform Host. Devices:\n'
b'2019-04-30 21:45:43.734441: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>\n'
b'2019-04-30 21:45:43.901689: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node 

In [15]:
#find newest model file
!ls /content/Mask_RCNN/logs/
from os import walk

logs = []
for (dirpath, dirnames, filenames) in walk("/content/Mask_RCNN/logs/"):
    logs.extend(dirnames)
    break

print(logs)
logs.sort(reverse=True) # get the most recent log
last_log = logs[0]

files = []
for (dirpath, dirnames, filenames) in walk("/content/Mask_RCNN/logs/"+last_log):
    files.extend(filenames)
    break
    
checkpoints = sorted(filter(lambda f: f.startswith("mask_rcnn"), files), reverse=True)
print(checkpoints)

last_model = checkpoints[0]
#drive.mount('/content/gdrive')


products20190430T2145
['products20190430T2145']
['mask_rcnn_products_0030.h5', 'mask_rcnn_products_0029.h5', 'mask_rcnn_products_0028.h5', 'mask_rcnn_products_0027.h5', 'mask_rcnn_products_0026.h5', 'mask_rcnn_products_0025.h5', 'mask_rcnn_products_0024.h5', 'mask_rcnn_products_0023.h5', 'mask_rcnn_products_0022.h5', 'mask_rcnn_products_0021.h5', 'mask_rcnn_products_0020.h5', 'mask_rcnn_products_0019.h5', 'mask_rcnn_products_0018.h5', 'mask_rcnn_products_0017.h5', 'mask_rcnn_products_0016.h5', 'mask_rcnn_products_0015.h5', 'mask_rcnn_products_0014.h5', 'mask_rcnn_products_0013.h5', 'mask_rcnn_products_0012.h5', 'mask_rcnn_products_0011.h5', 'mask_rcnn_products_0010.h5', 'mask_rcnn_products_0009.h5', 'mask_rcnn_products_0008.h5', 'mask_rcnn_products_0007.h5', 'mask_rcnn_products_0006.h5', 'mask_rcnn_products_0005.h5', 'mask_rcnn_products_0004.h5', 'mask_rcnn_products_0003.h5', 'mask_rcnn_products_0002.h5', 'mask_rcnn_products_0001.h5']


In [21]:
#download the model file
from google.colab import drive
#gdrive not working somehow
drive.mount('/content/gdrive')
print('cp /content/Mask_RCNN/logs/{0}/{1} "/content/gdrive/My Drive/{1}"'.format(last_log, last_model, last_model))
os.system('cp /content/Mask_RCNN/logs/{0}/{1} /content/gdrive/My Drive/{1}'.format(last_log, last_model, last_model))

#from google.colab import files
#files.download('/content/Mask_RCNN/logs/{0}/{1}'.format(last_log, last_model))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
cp /content/Mask_RCNN/logs/products20190430T2145/mask_rcnn_products_0030.h5 /content/gdrive/My Drive/mask_rcnn_products_0030.h5


256

In [0]:
#copy needs ""
#!cp /content/Mask_RCNN/logs/products20190430T2145/mask_rcnn_products_0030.h5 "/content/gdrive/My Drive/"
